## Load library modules

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

default_dir = os.path.dirname(os.path.dirname(os.getcwd()))
os.chdir(default_dir)

# (Mac) Uncomment below two lines if there's an error with PyJulia setup (initial execution may take some time)
# from julia.api import Julia
# jl = Julia(compiled_modules=False)

from modWorm import sys_paths
from modWorm import network_params as n_params
from modWorm import network_dynamics as n_dyn
from modWorm import network_interactions as n_inter
from modWorm import network_simulations as n_sim

from modWorm import muscle_body_params as mb_params
from modWorm import muscle_dynamics as m_dyn
from modWorm import body_dynamics as b_dyn
from modWorm import body_simulations as b_sim

from modWorm import proprioception_simulation as p_sim

from modWorm import utils
from modWorm import animation

### Load experimental connectome data
- Varshney, Lav R., et al. "Structural properties of the Caenorhabditis elegans neuronal network." PLoS computational biology 7.2 (2011): e1001066.
- Haspel, Gal, and Michael J. O'Donovan. "A perimotor framework reveals functional segmentation in the motoneuronal network controlling locomotion in Caenorhabditis elegans.
- Download link: https://www.wormatlas.org/images/NeuronConnect.xls

### Load neurons to muscles mapping data
- WormAtlas, Altun, Z.F., Herndon, L.A., Wolkow, C.A., Crocker, C., Lints, R. and Hall, D.H. (ed.s) 2002-2024.
- Download link: https://www.wormatlas.org/images/NeuronFixedPoints.xls
- If the file is in old .xls format, make sure to save as .xlsx format

In [ ]:
conn_gap, conn_syn = utils.construct_connectome_Varshney(filepath)
muscle_map = utils.construct_muscle_map_Hall(filepath)

## Define Nervous System

In [ ]:
from modWorm import predefined_classes_nv, predefined_classes_mb

celegans_nv = predefined_classes_nv.CelegansWorm_NervousSystem_PPC_Julia(conn_gap, conn_syn)
celegans_mb = predefined_classes_mb.CelegansWorm_MuscleBody_PPC_Julia(muscle_map)

### Load updated connectome data
- Cook, Steven J., et al. "Whole-animal connectomes of both Caenorhabditis elegans sexes." Nature 571.7763 (2019): 63-71.
- Haspel, Gal, and Michael J. O'Donovan. "A perimotor framework reveals functional segmentation in the motoneuronal network controlling locomotion in Caenorhabditis elegans.
- Download link: https://www.wormwiring.org/si/SI%205%20Connectome%20adjacency%20matrices,%20corrected%20July%202020.xlsx

In [ ]:
conn_gap_updated, conn_syn_updated = utils.construct_connectome_Cook(filepath)

## Modify model components

In [ ]:
# Using updated connectome data

celegans_nv.network_Electrical = n_inter.init_network_Electrical(conn_map = conn_gap_updated, 
                                                                 conductance_map = np.ones((279, 279)) * 0.1, 
                                                                 active_mask = np.ones(279, dtype = 'bool'))

celegans_nv.network_Chemical = n_inter.init_network_Chemical(conn_map = conn_syn_updated, 
                                                             conductance_map = np.ones((279, 279)) * 0.1,
                                                             polarity_map = n_params.CE.ei_map, 
                                                             active_mask = np.ones(279, dtype = 'bool'))

n_inter.init_vth_Linear(celegans_nv)

In [ ]:
# Alternative fluid viscosity parameter

celegans_mb.fluid_Viscosity = 0.001 # Default = 0.01 Ns/m^2

## Define Stimulation

In [ ]:
# We use the pulse of neural stimuli emulating the gentle touch to C.elegans posterior body region
# input_mat has dim = (1400, 279) (14 seconds)

gentle_posterior_stim = np.load('modWorm/presets_input/input_mat_gentle_post_touch.npy')

## Simulate Nervous System

In [ ]:
# Use run_network() function from "proprioception_simulation" module NOT "network_simulation" module

solution_dict_fwd = p_sim.run_network_julia(celegans_nv, celegans_mb, gentle_posterior_stim)

## Analyze results

In [ ]:
# Plot the body trajectory

plt.plot(solution_dict_fwd['x_solution'][:, 0], solution_dict_fwd['y_solution'][:, 0])
plt.xlim(-50, 50)
plt.ylim(-50, 50)

In [ ]:
# Animate the simulated worm body

animation.animate_body(x = solution_dict_fwd['x_solution'], y = solution_dict_fwd['y_solution'], filename = 'fwd_locomotion_ppc_custom',
                       xmin = -70, xmax = 20, ymin = -20, ymax = 70,
                       figsize_x = 10, figsize_y = 10, 
                       background_img_path = False, animation_config = mb_params.CE_animation)